[2026-01-23 Fiddler](https://thefiddler.substack.com/p/bingo)
====================

Fiddler
-------
There are 45! ways to order the numbers, but since the numbers not on my grid are
irrelevant, there are $8!$ ways to order the numbers.

There are 8 ways to get bingo after the first two numbers are marked, so the
probability of bingo after marking two numbers is $8\cdot6!/8!$.

There are 16 ways to have adjacent numbers marked after the first two numbers, and
there are 16 ways to have a corner and a non-adjacent number marked, there are
8 ways to have two adjacent non-corners marked, and there are 8 ways
to have two adjacent corners marked.

In the $16\cdot6!/8!$ chance of having adjacent numbers marked, there is a 3/6 probability
of bingo after marking three numbers, a 1/6 probability of having a corner and its
two adjacent numbers marked, meaing bingo is guaranteed after marking four numbers.
With the remaining 2/6, there is a 4/5 probability of bingo after marking four numbers
and 1/5 probability of bingo after marking five numbers.

In the $16\cdot6!/8!$ chance of having a corner and a non-adjacent number marked, there is
a 2/6 probability of bingo after marking three numbers, and 1/6 chance that the
third number is adjacent to the marked corner, 1/6 chance that the third number
is adjacent to the marked non-corner, 1/6 chance that the third number is the
remaining corner, and 1/6 chance that the third number is the remaining non-corner.
When the third number is adjacent to the marked corner, the case is the same as
when the first two numbers were adjacent and the third was a non-adjacent non-corner.

This is getting to be too much to think about, so I'll resort to code.

In [1]:
from functools import cache

@cache
def average_markers3(p, marked):
    avg = 0
    pn = p/(8-len(marked))
    for n in [1..8]:
        if n in marked:
            pass
        elif n == 1:
            if ((2 in marked and 3 in marked) or
                (4 in marked and 6 in marked) or
                8 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([1]))
        elif n == 2:
            if ((1 in marked and 3 in marked) or 7 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([2]))
        elif n == 3:
            if ((1 in marked and 2 in marked) or
                (5 in marked and 8 in marked) or
                6 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([3]))
        elif n == 4:
            if ((1 in marked and 6 in marked) or 5 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([4]))
        elif n == 5:
            if ((3 in marked and 8 in marked) or 4 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([5]))
        elif n == 6:
            if ((1 in marked and 4 in marked) or
                (7 in marked and 8 in marked) or
                3 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([6]))
        elif n == 7:
            if ((6 in marked and 8 in marked) or 2 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([7]))
        elif n == 8:
            if ((3 in marked and 5 in marked) or
                (6 in marked and 7 in marked) or
                1 in marked):
                avg += pn*(1+len(marked))
            else:
                avg += average_markers3(pn, marked | frozenset([8]))
    return avg

In [2]:
average_markers3(1, frozenset()), numerical_approx(average_markers3(1, frozenset()))

(243/70, 3.47142857142857)

Extra credit
------------
More code.

In [3]:
@cache
def average_markers(n, p, marked):
    avg = 0
    pn = p/(n*n - len(marked))
    for i in range(n):
        for j in range(n):
            if (i,j) in marked:
                pass
            else:
                new_marked = marked | frozenset([(i,j)])
                if all([(k,j) in new_marked for k in range(n)]):
                    avg += pn*len(marked)
                elif all([(i,k) in new_marked for k in range(n)]):
                    avg += pn*len(marked)
                elif i == j and all([(k,k) in new_marked for k in range(n)]):
                    avg += pn*len(marked)
                elif i == n - 1 - j and all([(k,n-1-k) in new_marked for k in range(n)]):
                    avg += pn*len(marked)
                else:
                    avg += average_markers(n, pn, new_marked)
    return avg

In [4]:
result = average_markers(3, 1, frozenset([(1,1)]))
result, numerical_approx(result)

(243/70, 3.47142857142857)

In [5]:
# this is super slow: result = average_markers(5, 1, frozenset([(2,2)]))
result = 4245967/312018
result, numerical_approx(result)

(4245967/312018, 13.6080835080027)

Numerical simulations
---------------------
[Numerical simulations](20260123.go) agree:

    $ go run 20260123.go
    3-by-3, 800000 trials: 3.470803 markers
    5-by-5, 800000 trials: 13.604291 markers
    3-by-3, 800000 trials: 17.746680 calls, 3.471909 markers
    5-by-5, 800000 trials: 41.383085 calls, 13.607770 markers
    3-by-3, 8000000 trials: 3.472149 markers
    5-by-5, 8000000 trials: 13.606783 markers
    3-by-3, 8000000 trials: 17.736121 calls, 3.471165 markers
    5-by-5, 8000000 trials: 41.368968 calls, 13.607681 markers
    3-by-3, 80000000 trials: 3.471211 markers
    5-by-5, 80000000 trials: 13.608461 markers
    3-by-3, 80000000 trials: 17.742934 calls, 3.471474 markers
    5-by-5, 80000000 trials: 41.368508 calls, 13.607828 markers

Further thoughts
----------------
I originally thought the problem was the average number of numbers called before bingo rather than
the number of numbers marked, which is slightly more complicated.  But the code is too slow for the
5-by-5 grid.

The result of $621/35 \approx 17.743$ numbers called on average before bingo for the 3-by-3 grid
agrees with the results of the numerical simulations.

In [6]:
@cache
def average_calls(n, p, calls, marked, uncalled):
    if p == 0:
        return 0, 0
    if (any([all([(i,j) in marked for j in range(n)]) for i in range(n)])
        or any([all([(j,i) in marked for j in range(n)]) for i in range(n)])
        or all([(i,i) in marked for i in range(n)])
        or all([(i,n-1-i) in marked for i in range(n)])):
        return p*calls, p*(len(marked)-1)
    avg_calls = 0
    avg_marked = 0
    pn = p/uncalled
    unmarked = 0
    for i in range(n):
        for j in range(n):
            if (i,j) not in marked:
                unmarked += 1
                new_marked = marked | frozenset([(i,j)])
                p_calls, p_marked = average_calls(n, pn, calls+1, new_marked, uncalled-1)
                avg_calls += p_calls
                avg_marked += p_marked
    p_calls, p_marked = average_calls(n, pn*(uncalled-unmarked), calls+1, marked, uncalled-1)
    avg_calls += p_calls
    avg_marked += p_marked
    return avg_calls, avg_marked

In [7]:
result = average_calls(3, 1, 0, frozenset([(1,1)]), 45)
result, (numerical_approx(result[0]),numerical_approx(result[1]))

((621/35, 243/70), (17.7428571428571, 3.47142857142857))

In [8]:
# this is too slow and uses too much memory
# result = average_calls(5, 1, 0, frozenset([(2,2)]), 75)
# result, (numerical_approx(result[0]),numerical_approx(result[1]))